In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

## Langchain tracking and tracing
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI

google_genai_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=1.9);
response = google_genai_llm.invoke("Tell me a good joke about cars");
print(response.content)

Why did the car get a flat tire?

Because it ran over a bunch of nails!  And then it went to a nail salon... it needed a manicure!


## PROMPTS

In [5]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

str_ouput_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a famous standup comic."),
    ("user","{input}")
])

chain = prompt | google_genai_llm | str_ouput_parser
response = chain.invoke({"input":"Tell a joke on cars"})
print(response)

I bought a new car the other day.  It’s so self-driving, I just sit there and tell it where to go.  The problem is, it takes my directions *way* too literally.  I told it to take me to the nearest comedy club...  It ended up parked outside a clown college.  Apparently, it has a very loose interpretation of "funny."


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser


json_ouput_parser = JsonOutputParser()

format_instructions = json_ouput_parser.get_format_instructions() # "Returns a JSON Object"

prompt2 = ChatPromptTemplate.from_messages([
    ("system", "Your name is {name}. Return a JSON object. {format_instructions}."), # Inject format instructions
    ("user","The JSON should have a 'joke' key and a 'punchline' key. If joke not allowed make key answer with not alloed"),
    ("ai", "sure"),
    ("user", "{input}"),
])

# Partially binding
prompt_with_instructions = prompt2.partial(format_instructions=format_instructions)

chain = prompt_with_instructions | google_genai_llm | json_ouput_parser

response = chain.invoke({
    "name": "Rahul",
    "input": "Tell a joke on cars",
})
print(response)

{'joke': 'Why did the car get a flat tire?', 'punchline': 'Because it ran over a bunch of nails!'}


# Pydantic Integration

In [30]:
from pydantic import BaseModel,Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

class Joke(BaseModel):
    setup:str = Field(description="This is question setting up the joke. If joke no allowed keep it empty.")
    punchline:str = Field(description="Answer to resolve the joke. If joke no allowed keep it empty.")
    allowed:bool = Field(description="If the joke is not allowed keep it false else true")

json_op = JsonOutputParser(pydantic_object=Joke)
prompt = PromptTemplate(
    template="""You are the world's best standup comic.
    IMPORTANT RULE: Jokes on vegans are ABSOLUTELY NOT ALLOWED.
    If the user asks for a joke about vegans or someting other offensive, you MUST respond with an empty setup, empty punchline, and set 'allowed' to false.
    For any other joke, provide a setup and punchline, and set 'allowed' to true.

    Query: {query}
    Output Format: {output_format}
    """,
    input_variables=["{query}"],
    partial_variables={"output_format":json_op.get_format_instructions()}
)

chain = prompt | google_genai_llm | json_op
response = chain.invoke({"query":"Make a joke on students looks jobs to ai"})
response


{'setup': "I saw a student's resume that listed 'proficient in AI' under their skills.  I asked them what AI tools they used...",
 'punchline': "...they said, 'I use AI to look for jobs.'",
 'allowed': True}